In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
item_name = ['Power Cord', 'Phone Charger', 'Ear Buds', 'Mouse' ,'Keyboard', 'Milk', 'Eggs', 'Cereal', 'Shrimp',
             'Noodles', 'Steak', 'King Crab', 'Tape', 'Glue', 'Nails', 'Bracket', 'Brush', 'Paint']

- train_data : 2015 1분기 ~ 2021 4분기까지 data (industry data 기준 feature)

- label_data 와 label_plus : 2015 2분기 ~ 2022년 1분기까지 data (retail sale data 'Sale')

(데이터를 따로 저장해둬서, 본 코드에서 합쳐서 사용)

- test_data : 2022년 1분기 data (industry data 기준)

> "직전분기 industry data 기준으로 다음 분기 retail sale 예측하기"

In [254]:
for iname in item_name:
    train_data = pd.read_csv('../input/item-info/'+iname+'_quarter_total_sale.csv',encoding = 'cp949')
    label_data = pd.read_csv('../input/item-info/'+iname+'_quarter_label.csv',encoding = 'cp949')
    label_plus = pd.read_csv('../input/testingset/test_'+iname+'_quarter_label.csv',encoding = 'cp949')
    test_data = pd.read_csv('../input/testingset/test_'+iname+'_quarter_total_sale.csv', encoding ='cp949')

    label_set = pd.concat([label_data,label_plus]) # 학습 data label
    label_set = label_set.reset_index()
    label_set = label_set.drop(['index'],axis=1)
    
    #filter = train_data[train_data['year']<2020] # 직전년도 data만 사용해서 학습할 때
    #train_data = train_data.drop(tfilter.index)
    #lfilter = label_set[label_set['year']<2020]
    #label_set = label_set.drop(lfilter.index)
    
    label_del = label_set[label_set['year']==2015] #현재 label data에는 2015년 1분기부터 존재 -> 따라서 제거하기
    label_d= label_del[label_del['quarter']==1]
    sample_label = label_set.drop(label_d.index)
    

    sort_train = train_data.sort_values(['year','quarter','store']) # 순서에 맞춰 sorting
    sort_test = test_data.sort_values(['year','quarter','store'])

    sample_train_data = sort_train.reset_index()
    sample_label_data = sample_label.reset_index()

    sample_train_data = sample_train_data.drop(['index'],axis=1)
    sample_label_data = sample_label_data.drop(['index'],axis=1)

    sample_label_data = sample_label_data.sort_values(['year','quarter','store'])

    label_column = sample_label_data['sale']

    sample_train_data = sample_train_data.drop(['type','item','store'],axis=1) 
    # item별로 학습하기 때문에 type과 item은 무의미한 feature
    # store의 경우 store 지역 위치, urban/rural 정보, 과거 판매량에 해당하는 정보로 대체
    
    sample_test_data = sort_test.drop(['type','item','store'],axis=1)

    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    sample_train_data['year'] = encoder.fit_transform(sample_train_data['year']) # year의 경우, 동일한 년도와 흐름 정보만 있으면 되므로 label encoder 진행

    sample_test_data['year']= 7 # test data에만 2022년이 존재하므로 7로 변환 

    from sklearn.preprocessing import MinMaxScaler # 전체 feature들 scale 맞추기
    scaler = MinMaxScaler()
    sample_train_data= scaler.fit_transform(sample_train_data)
    sample_test_data= scaler.transform(sample_test_data)

    from sklearn.linear_model import LinearRegression # 모델 학습
    model = LinearRegression()
    model.fit(sample_train_data, label_column)
    y_pred=model.predict(sample_test_data)

    result_file = pd.read_csv('../input/submission/sampling_file.csv', encoding ='cp949') # 정답제출할 파일 

    rfilter = result_file['Item'] == iname #현재 정답파일은 모든 item에 모든 store에 대한 파일 -> 해당 item에 대한 부분만 추출
    r_frame = result_file[rfilter]
    result = r_frame.reset_index()
    result = result.drop(['index'],axis=1)
    result = result.sort_values(['Store']) #학습에 사용한 파일에 맞춰 sorting

    s_predict= np.around(y_pred) # 소수점으로 나오는 경우 많으므로, 정수로 반올림

    result['Allocation'] = s_predict # 예측결과, allocation column에 저장

    result.to_csv(iname+'_allocation.csv',index=False) # 정답 csv로 내보내기

In [279]:
item_name = ['Power Cord', 'Phone Charger', 'Ear Buds', 'Mouse' ,'Keyboard', 'Milk', 'Eggs', 'Cereal', 'Shrimp',
             'Noodles', 'Steak', 'King Crab', 'Tape', 'Glue', 'Nails', 'Bracket', 'Brush', 'Paint']

A_category = ['Phone Charger','Keyboard','Shrimp']
B_category = ['Ear Buds', 'Noodles', 'Steak', 'King Crab']

example_test = pd.DataFrame(columns=['Allocation','Store','Item'])

for iname in item_name:
    r_item = pd.read_csv('./'+iname+'_allocation.csv',encoding = 'cp949') # item마다 allocation 결과 파일 불러오기
    r_item.loc[r_item['Allocation'] <0 , 'Allocation'] = 0  #음수는 불가능하므로, 0으로 변환
    total = r_item['Allocation'].sum() # 총량 계산
    r_item['Allocation'] = r_item['Allocation']/total # 각 store 비율 계산
    if iname in A_category: # 카테고리별 실제 allocation 량 계산 (business importance에 따라 총량을 다르게 함)
        r_item['Allocation'] = r_item['Allocation']*500
    elif iname in B_category:
        r_item['Allocation'] = r_item['Allocation']*250
    else:
        r_item['Allocation'] = r_item['Allocation']*10
        
    r_item['Allocation']= np.around(r_item['Allocation']) # 정수로 변환
    example_test = pd.concat([example_test, r_item], ignore_index=True) # item마다 결과 합치기

In [ ]:
print(example_test['Allocation'].sum()) # allocation 총량 확인하는 과정

In [281]:
example_test.to_csv('allocation_file.csv',index=False) #최종 결과 csv 내보내기